In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import pandas as pd

# Specify your file path
file_path = '../crawling/file_content_list.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)
df = df.dropna(subset=['Content'])

# Display the first few rows of the DataFrame
print(df.head(10))


                                               Title  Category        Date  \
0                           SKKU AI Colloquium  2023      [기타]  2023-09-14   
1                     실감미디어공학과 세미나 안내 (9/14, 10:30~)  [행사/세미나]  2023-09-12   
2                     [공개SW특강] 대규모 오픈소스 프로그램 쉽게 분석하기  [행사/세미나]  2023-09-12   
3                           2023 국방 AI 경진대회 (MAICON)      [기타]  2023-09-11   
4             지능형소프트웨어학과(대학원) 진학상담회 및 산학장학생 안내(9/18)  [행사/세미나]  2023-09-11   
5                               2023년-2학기 실험실습 TA 모집   [채용/모집]  2023-09-08   
6               디지털인재얼라이언스 가입 기업 대상 인턴십 희망 기업 안내 리스트   [채용/모집]  2023-09-07   
7                        2023년-2학기 1만라인 TA 모집(~9/12)   [채용/모집]  2023-09-06   
8                                   [SPARK 12기 참가모집]   [채용/모집]  2023-09-01   
9  [성균웹진] 미국 교환학생 후기 - The University of Texas at...      [기타]  2023-08-30   

                                             Content  \
0  SKKU AI Colloquium 2023Talking Sharing and Lea...   
1  실감미디어공학과 세미나가 아래와 같이 개최되오니

In [8]:
import numpy as np
import torch
from transformers import BertTokenizer, BertModel
import faiss

In [9]:
# 2. Vectorize Korean text columns using KoBERT
class KoBertEmbedding:
    def __init__(self):
        self.tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
        self.model = BertModel.from_pretrained('monologg/kobert')
        self.model.eval()

    def get_embedding(self, text):
        inputs = self.tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
        with torch.no_grad():
            outputs = self.model(**inputs)
        return outputs['last_hidden_state'][:,0,:].numpy()


In [12]:
print(df["Content"])

실감미디어공학과 세미나가 아래와 같이 개최되오니 많은 참여 부탁드립니다일시  9월 14일 목 1330장소  국제관 9B312호강연제목  Understanding Human in the 3D Space강연자  문경식 MetaPostdoctoral Research ScientistZOOM  ID 591 389 1581  PW 908353


In [27]:
# Vectorize using KoBERT
kobert = KoBertEmbedding()

#raws Content를 바탕으로 한 비교는 의미있는 뚜렷한 결과를 양산하지는 못하고 있음
#df["Vectors"] = df["Content"].apply(kobert.get_embedding)
df["Vectors"] = df["Title"].apply(kobert.get_embedding)
vectors = np.vstack(df["Vectors"].values)

# 3. Create a Faiss index and add the vectors
d = vectors.shape[1]
index = faiss.IndexFlatL2(d) 
index.add(vectors)

In [29]:
def search_k_nearest(text, K=10):
    vector = kobert.get_embedding(text)
    distances, indices = index.search(vector, K)
    for i in range(K):
        print(f"Rank {i+1}: {df.iloc[indices[0][i]]['Title']} (Distance: {distances[0][i]:.4f})")


In [49]:
search_k_nearest("기말고사 대비")

Rank 1: 제4회 대학생 AI x BOOKATHON 대회 (Distance: 41.9828)
Rank 2: 2023 인공지능대학원 심포지원 (Distance: 42.6053)
Rank 3: 2023 공개SW 개발자대회 (Distance: 42.6053)
Rank 4: 2023학년도 학생건강검진 시행 (Distance: 42.6053)
Rank 5: SW개발자 취업준비생을 위한 채용 페스티벌 행사 (Distance: 43.1535)
Rank 6: (연장) 2022년도 하계집중근무 모집 (Distance: 44.4828)
Rank 7: 2023학년도 2학기 데이터분석기초(인사캠) TA 모집 (Distance: 46.2151)
Rank 8: 2023학년도 2학기 AI기초와활용(자과캠) 실습TA 모집 (Distance: 46.2151)
Rank 9: 2023학년도 1학기 AI기초와활용(자과캠) 실습TA 모집 (Distance: 46.2151)
Rank 10: 2021 헬스허브 데이터톤(2021 Healthhub Datathon) 대회 (Distance: 46.2567)
